In [1]:
#colab 용 konlpy 모듈
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

--2023-05-05 04:47:02--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2023-05-05 04:47:02--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucdf3617561288be1809f546e2d2.dl.dropboxusercontent.com/cd/0/get/B7f7-mmlO44g3bRzjrslcUpxVAZcWq6uHaO1_Nq_gjJ3Dj7bJ_2J9RZiTa4MbhnGC8oy5DlqX9GrwdE6UICiPx8YtEKc_XNoG7CaG_sCLV5UM33wDPKa3hAZQSVYUyAA2gUNMac8aJ4TLtRjT2pq40HXthupoOwiJYIfTZGeAv0D_jyzA7IqboiwUiG6-bIW2Nk/file?dl=1# [following]
--2023-05-05 04:47:02--  https://ucdf3617561288be1809f546e2d2.dl.dropboxusercontent.com/cd/0/get/B7f7-mmlO44g

In [ ]:
# 기존 word2vec 로 학습한 모델을 doc2vec 기반의 모델로 변경후 d2v 이라는 이름으로 저장
import ssl
import certifi
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from konlpy.tag import Okt
from tqdm import tqdm

train_data = pd.read_csv('/content/sample_data/news.csv')
print(train_data)
train_data['document'] = train_data['title']+' '+train_data['contents']
train_data['label'] = train_data['keyword'].map({'연애': 0, '스포츠': 1, '정치': 2, '국제': 3, '사회': 4, '문화': 5})
print(train_data.isnull().values.any())
train_data = train_data.dropna(how='any')
print(len(train_data))

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','불펌','금지','기사','기자','속보']
okt = Okt()

# Doc2Vec 학습을 위해 TaggedDocument 생성
tagged_data = [TaggedDocument(words=okt.morphs(_d, stem=True), tags=[str(i)]) for i, _d in enumerate(tqdm(train_data['document']))]

model = Doc2Vec(tagged_data, vector_size=100, window=5, min_count=5, workers=6, dm=1)

model.save('review_d2v.model')

       idx                                           title  \
0        1                 보라 데보라 유인나윤현민 로코 장인들의 연애 루저 탈출기   
1        2                          홍김동전 주우재홍진경 극과 극 연애 조언   
2        3                    유퀴즈환승연애 제작진 뭉쳐 대 연애예능 소소연 론칭   
3        4             한혜진 살 차이상사와 비밀 연애 악조건 다 갖고 있다 연애의참견   
4        5                       환승연애  대 리얼리티 제작소년 소녀 연애하다   
...    ...                                             ...   
7446  7447                   서울 코리아오픈 국제휠체어테니스 대회 일간 열전 마쳐   
7447  7448                    한은 국제유가 상방압력 우세  수요회복 수준이 관건   
7448  7449                                  국제 설탕값 년 만에 최고   
7449  7450                     동대구역 광장 공공디자인 국제 디자인 어워즈 수상   
7450  7451  유류세 조정 찬물 끼얹은 국제유가 세수물가 동시타격 세수부족유가상승 고심 깊은 정부   

                                               contents keyword  
0     마이데일리  노한빈 기자  새 오리지널 드라마 보라 데보라연출 이태곤 서민정 극본 ...      연애  
1      예능 홍김동전연출 박인석은 홍진경 김숙 조세호 주우재 우영이 출연해 동전으로 운명...      연애  
2     서울뉴스 윤효정 기자  대들의 연애 리얼리티 소소연이 론칭된다 엔터테인먼트 스트리밍...  

100%|██████████| 7451/7451 [13:26<00:00,  9.24it/s]


In [ ]:
import ssl
import certifi
import pandas as pd
import urllib.request
from gensim.models.doc2vec import Doc2Vec
from konlpy.tag import Okt
from tqdm import tqdm

# 데이터 로드
news_data = pd.read_csv('/content/sample_data/news.csv')

# 데이터 전처리
news_data['document'] = news_data['title'] + ' ' + news_data['contents']
news_data['label'] = news_data['keyword'].map({'연애': 0, '스포츠': 1, '정치': 2, '국제': 3, '사회': 4, '문화': 5})
news_data = news_data.dropna(how='any')

# # 불용어(stopwords)와 형태소 분석기(okt) 준비
# stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','불펌','금지','기사','기자','속보']
# okt = Okt()

# Doc2Vec 모델 로드
model = Doc2Vec.load('review_d2v.model')

# 추천 할 문서 갯수 선정 
num_recommendations = 5

# 추천할 문서 선택 (데이터프레임의 index 기준으로)
consumed_doc_idx = 1

# 선택한 문서 출력
print(news_data['title'].iloc[consumed_doc_idx])
print(news_data['contents'].iloc[consumed_doc_idx])

# 선택한 문서를 기반으로 유사한 문서들 추천
similar_docs = model.dv.most_similar(consumed_doc_idx, topn=num_recommendations)

print(similar_docs)



홍김동전 주우재홍진경 극과 극 연애 조언
 예능 홍김동전연출 박인석은 홍진경 김숙 조세호 주우재 우영이 출연해 동전으로 운명을 체인지하는 피땀눈물의 구개념 버라이어티 홍김동전은 국내  플랫폼 웨이브에서 지난해 전체 프로그램 위를 시작으로 위를 거쳐 최근 위를 기록한 데 이어 웨이브 신규유료가입 견인 콘텐츠 예능 부문 위를 차지했다 또한 굿데이터코퍼레이션에 따르면 홍김동전은 월 주 차  통합 화제성 비드라마 부문 위 홍김동전에 출연한 방탄소년단 지민은  통합 화제성 비드라마 출연자 부문 위에 오르며 연일 화제가 되고 있다오는 일 방송되는 홍김동전 회는  예능 스타 골든벨을 오마주한 동전 골든벨 탄으로 꾸며진다 고등학교 학창 시절로 돌아간 멤버들은 부산 창원 등 각자의 모교 후배들과 한 팀이 되어 동전 골든벨을 펼친다 그런 가운데 주우재가 여 고교 후배의 썸 갈등 사연에 자신의 산전수전 경험을 토대로 조언을 한다홍김동전 멤버들이 고교 후배들의 생생한 연애의 참견에 나선다 이 중 년 동안 애매하게 연락을 이어가는 여고생의 썸 사연이 등장하자 연애도 남사친도 아닌 애매한 썸 관계에 홍김동전 경력직 선배들의 조언이 무수히 쏟아졌다는 전언이다특히 주우재는 연애 조언 경력자답게 사연을 듣자마자 제대로 된 남자는 좋아하는 여자를 헷갈리게 하지 않는다며 프로 참견러의 면모를 발산해 시선을 사로잡는다 주우재는 장담하는데 그 남자는 여러 여자에게 같은 톡을 보냈을 가능성이 크다 절대 만나지 마라고 매운맛 팩폭을 이어간다 과거 첫사랑 때문에 일주일 동안 밥을 먹지 않았던 주우재인 만큼 경험에서 우러나오는 조언이 후배의 정신을 혼미하게 만들었다는 후문 김숙 역시 주우재의 말에  공감하며 후배들을 위해 조언을 아끼지 않는다고 해 주우재가 프로 연애 참견러로서 어떤 활약을 펼쳤을지 궁금증이 모인다하지만 그런 주우재에게 반기를 든 멤버가 있었으니 바로 홍진경 홍진경은 썸남이 있다는 자체가 소중한 일이라며 주우재와 도 다른 연애 참견으로 후배들의 공감을 유발한다 특히 홍진경은 지금은 모른다

# 새 섹션

In [ ]:
for doc_idx, sim in similar_docs:
        row = news_data.iloc[int(doc_idx)]
        doc_title = row['title']
        doc_keyword = row['keyword']
        doc_content = row['contents']
        print(f"유사도: {sim}, 제목: {doc_title}, 카테고리: {doc_keyword}, 본문: {doc_content}")

유사도: 0.6605522632598877, 제목: 자칭 연애고수 주우재 은근한 플러팅 당해 좋아하는 향 묻더니 홍김동전, 카테고리: 연애, 본문: 마이데일리  노한빈 기자 모델 주우재가 향수로 플러팅 당한 경험을 고백한다일 방송되는   홍김동전연출 박인석 회는  예능 스타 골든벨을 오마주한 동전 골든벨 탄이 그려진다 학창 시절로 돌아간 멤버들이 서울 부산 창원 등 각지에서 직접 찾아온 모교 후배들과 한 팀이 되어 동전 골든벨을 펼친다 그런 가운데 주우재가 자신이 좋아하는 향으로 플러팅 당한 경험을 밝혀 현장을 달아오르게 만든다이날 홍김동전 멤버들과 모교 후배들이 연애에 대한 허심탄회한 이야기를 나눈다 이때 주우재가 은근하게 플러팅 하는 방법을 알고 있다고 말해 이목을 집중시킨다주우재는 내가 당했던 플러팅이라고 운을 뗀 뒤 상대가 어느 날 좋아하는 향이 무엇이냐고 물어보더라 그런데 다음 만남에서 상대에게 내가 좋아하는 향이 났다라고 설렘 가득했던 당시를 회상한다 이어 주우재는 여기서 포인트는 무심하듯 시크하게 물어보는 것이라며 스킬까지 공개해 현장을 달달하게 만들었다이와 함께 주우재는 연애 플러팅 필살기를 묻는 질문에 대해 가장 담백한 게 베스트라고 전한다 자칭 연애 고수다운 면모를 선보인 주우재는 멤버들의 따가운 시선을 피할 수 없었다고 한다특히 주우재의 향수 플러팅뿐만 아니라 신박한 고백 방법이 공개된다고 해 귀추가 주목된다 세대의 에어드롭 고백부터 세대 추억의 고백까지 각양각색의 플러팅에 학구열을 불태웠다는 전언이다 과연 후배들의 심장을 저격한 홍김동전 최고의 연애 고수는 누구일지 궁금증이 고조된다한편 홍김동전은 매주 목요일 오후 시 분 방송된다홍김동전 출연하는 주우재 사진   제공 마이데일리 무단전재재배포 금지 
유사도: 0.622671902179718, 제목: 추성훈  정국과 스파링 하는 사이정국 복싱 실력 공개전참시, 카테고리: 연애, 본문: 일 방송되는  예능 프로그램 전지적 참견 시점기획 박정규  연출 김윤집 이준범 이경순 김해니  작가 여현전 

In [3]:
# 기존 word2vec 로 학습한 모델을 doc2vec 기반의 모델로 변경후 d2v 이라는 이름으로 저장
import ssl
import certifi
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from konlpy.tag import Mecab
from tqdm import tqdm

train_data = pd.read_csv('/content/sample_data/news.csv')
print(train_data)
train_data['document'] = train_data['title']+' '+train_data['contents']
train_data['label'] = train_data['keyword'].map({'연애': 0, '스포츠': 1, '정치': 2, '국제': 3, '사회': 4, '문화': 5})
print(train_data.isnull().values.any())
train_data = train_data.dropna(how='any')
print(len(train_data))

# okt to mecab 
mecab = Mecab()

stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다', '불펌', '금지', '기사', '기자', '속보']
tagged_data = [TaggedDocument(words=mecab.morphs(_d), tags=[str(i)]) for i, _d in enumerate(tqdm(train_data['document']))]

model = Doc2Vec(tagged_data, vector_size=100, window=5, min_count=5, workers=6, dm=1)
model.save('review_d2v.model')

         idx                                    title  \
0          1             유퀴즈환승연애 제작진 뭉쳐 대 연애예능 소소연 론칭   
1          2      한혜진 살 차이상사와 비밀 연애 악조건 다 갖고 있다 연애의참견   
2          3         환승연애 유퀴즈 작가 뭉쳤다소년 소녀 연애하다 하반기 공개   
3          4            환승연애유퀴즈 제작진 의기투합티빙 소년 소녀 연애하다   
4          5  결국 최종 목표는 연예인 데뷔였나연애 말고 연예인이 좋은 리얼리티 피플   
...      ...                                      ...   
10274  10275       방탄소년단 제이홉 군대 식단 전세계 팬들에게 한식 관심 이어져   
10275  10276           포토 바이랩  브랜드 고객충성도 대상 펫샴푸 부문 수상   
10276  10277       포토 비에날퀸  브랜드 고객충성도 대상 갱년기유산균 부문 수상   
10277  10278      포토 일렉트로룩스  브랜드 고객충성도 대상 식기세척기 부문 수상   
10278  10279            포토 킨토  브랜드 고객충성도 대상 텀블러 부문 수상   

                                                contents keyword  
0      서울뉴스 윤효정 기자   대들의 연애 리얼리티 소소연이 론칭된다 엔터테인먼트 스트리...      연애  
1      엑스포츠뉴스 김유진 기자 연애의 참견 한혜진이 살 차이 상사와 연애하는 고민녀의 사...      연애  
2      티빙 측은 일 오리지널 시리즈 소년 소녀 연애하다극본 이언주연출 박희연가 오는 하반...      연애  
3      대들의 성장담을 그린 리얼리티 예능 프로그램이 공개된다 일 티빙 측은 대

100%|██████████| 9302/9302 [00:40<00:00, 229.15it/s]


In [8]:
import pandas as pd
# import urllib.request
from gensim.models.doc2vec import Doc2Vec

# 데이터 로드
# news_data = pd.read_csv('/content/sample_data/news.csv')

# 데이터 전처리
# news_data['document'] = news_data['title'] + ' ' + news_data['contents']
# news_data['label'] = news_data['keyword'].map({'연애': 0, '스포츠': 1, '정치': 2, '국제': 3, '사회': 4, '문화': 5})
# news_data = news_data.dropna(how='any')

# Doc2Vec 모델 로드
model = Doc2Vec.load('review_d2v.model')

# 추천 할 문서 갯수 선정 
num_recommendations = 100

# 추천할 문서 선택 (데이터프레임의 index 기준으로)
consumed_doc_idx = 231

# 선택한 문서 출력
# print(news_data['title'].iloc[consumed_doc_idx])
# print(news_data['contents'].iloc[consumed_doc_idx])

# 선택한 문서를 기반으로 유사한 문서들 추천
similar_docs = model.dv.most_similar(consumed_doc_idx, topn=num_recommendations)

print(similar_docs)



[('177', 0.962590217590332), ('187', 0.8634777665138245), ('229', 0.8397485017776489), ('124', 0.8028554320335388), ('190', 0.7974991798400879), ('6860', 0.7831932902336121), ('7951', 0.77888423204422), ('55', 0.7743939757347107), ('778', 0.7719213366508484), ('753', 0.7708947062492371), ('665', 0.7703089714050293), ('363', 0.770148515701294), ('6905', 0.767137885093689), ('721', 0.7669379711151123), ('4569', 0.7656305432319641), ('88', 0.7652179002761841), ('2831', 0.7632558345794678), ('6851', 0.7632305026054382), ('3654', 0.763200044631958), ('4639', 0.7629187703132629), ('412', 0.7608544826507568), ('6878', 0.7606484889984131), ('6896', 0.7598777413368225), ('6815', 0.759371280670166), ('6869', 0.7593374848365784), ('503', 0.7572125792503357), ('737', 0.7565289735794067), ('4599', 0.7564593553543091), ('6909', 0.7563620209693909), ('7952', 0.7556807994842529), ('3609', 0.7556493878364563), ('8614', 0.7553770542144775), ('745', 0.7548869252204895), ('6915', 0.7544553875923157), ('35

In [9]:
for doc_idx, sim in similar_docs:
        row = news_data.iloc[int(doc_idx)]
        doc_title = row['title']
        doc_keyword = row['keyword']
        doc_content = row['contents']
        print(f"유사도: {sim} \n, 제목: {doc_title} \n, 카테고리: {doc_keyword} \n, 본문: {doc_content} \n")

유사도: 0.962590217590332 
, 제목: 김종국 아이브에 연애 상담 장원영 대답에 되게 쿨하다 감탄 짐종국 
, 카테고리: 연애 
, 본문: 마이데일리  임유리 기자 김종국이 아이브에게 연애 상담을 했다 일 유튜브 채널 짐종국에는 그룹 아이브가 출연한 영상이 게재됐다 영상에서 김종국은 아이브에게 남자가 데이트를 해야 되는데 그날 운동을 못 했다 그러고는 초조해하면 어떻게 할 거냐고 물었다 이에 장원영은 그럼 잠깐 하고 와 나랑 있을 때 이렇게 초조해하느니 잠깐 한 시간 하고 와 할 거 같다라고 대답했다 장원영의 대답에 김종국은 되게 쿨하다라며 감탄했다 제작진이 근데 그렇게 하고 축구를 가야 된다고 하면 어떻게 할 거냐고 묻자 장원영은 그러면 그거는이라면 고개를 저었다 가을이 그런 적이 혹시 있으시냐고 묻자 김종국은 그런데 진짜 약간 화가 나니 그러면이라고 되물었다 김종국의 진지한 모습에 리즈는 진심으로 물어보신다라며 폭소했다 김종국이 아니 그런데 다른 나쁜 행동을 하는 거는 아니지 않냐 운동하러 가는 거다라며 설명하자 리즈는 만약에 연인 사이면 같이 보내는 시간이 굉장히 중요한데 머릿속에 운동이 있고 축구가 있고 그러면 좀 그럴 수 있다라고 설명했다 김종국이 그런데 축구가 맨날 정해져 있다 일주일에 딱 그날이다 그러면 이해하지 않을까라고 묻자 장원영은 이해해 주시는 분을 만나면 좋을 거 같다라고 대답해 웃음을 자아냈다 사진  유튜브 채널 짐종국 영상 캡처  마이데일리 무단전재재배포 금지  

유사도: 0.8634777665138245 
, 제목: 김종국 아이브에 연애 상담 장원영 대답에 되게 쿨하다 감탄 
, 카테고리: 연애 
, 본문: 머니투데이 이은 기자 가수 김종국이 그룹 아이브에게 연애 상담을 했다지난 일 유튜브 채널 짐종국에는 아이브가 운동하는 귀한 영상이라는 제목의 영상이 공개됐다 이 영상에서 김종국은 남자가 데이트를 해야 되는데 그날 운동을 못 했다 그러고는 초조해하면 어떻게 할 거냐고 물었다이에 장원영은 그럼 잠깐 하고 와 나랑 있을

In [18]:
import pandas as pd
from konlpy.tag import Mecab
from gensim.models.doc2vec import Doc2Vec
import numpy as np

def recommend_documents(keyword, n=5):
    mecab = Mecab()
    model = Doc2Vec.load("review_d2v.model")

    # 입력된 키워드를 형태소 분석하여 doc2vec 모델에서 유사한 벡터 추출 (mecab 형태소 분석 모델 사용)
    keyword_vector = model.infer_vector(mecab.morphs(keyword))

    # 모든 문서와의 유사도 계산 
    doc_vectors = [model.docvecs[str(i)] for i in range(len(model.docvecs))]
    # numpy 모듈의 np.dot() 메서드를 사용하여 학습된 모든 문서 벡터 + 입력된 키워드간 내적 계산 
    similarities = np.dot(doc_vectors, keyword_vector)

    # 상위 n(5)개 문서 추출하여 반환
    top_n_indices = np.argsort(similarities)[::-1][:n]
    recommended_documents = pd.DataFrame(train_data.iloc[top_n_indices])
    return recommended_documents
  
#키워드 '북한'을 입력시 입력한 키워드와 학습된 모든 문서 벡터를 기반으로 유사한 문서 추천 테스트
recommend_documents('북한')

<ipython-input-18-c75068c098d3>:14: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  doc_vectors = [model.docvecs[str(i)] for i in range(len(model.docvecs))]


,idx,title,contents,keyword,document,label
4658,4659,한국 드라마 마약 같아주애는 김정은 후계자문지방,왜 그는 목숨까지 걸고 북한에서 벗어나고 싶었을까요 배경만 놓고 보면 이유를 찾기 ...,연애,한국 드라마 마약 같아주애는 김정은 후계자문지방 왜 그는 목숨까지 걸고 북한에서 벗...,0.0
8842,8843,전문 대통령 상하원 합동회의 연설문,윤석열 대통령은 일현지시각 오전 미 상하원 합동회의에서 연설을 했다 한국 대통령의 ...,사회,전문 대통령 상하원 합동회의 연설문 윤석열 대통령은 일현지시각 오전 미 상하원 합...,4.0
5185,5186,미끼 허성태 복수하러 한국 왔다장근석에 거래합시다종합,김나연 기자 미끼 허성태가 장근석에 거래를 제안했다일 오후 쿠팡플레이 오리지널 드라...,연애,미끼 허성태 복수하러 한국 왔다장근석에 거래합시다종합 김나연 기자 미끼 허성태가 장...,0.0
4295,4296,그레고리 헨더슨과 지적 동반 여행을 떠난 까닭,이현휘 한국교원대 인문사회과학교육연구소 연구원 내가 헨더슨 평전을 쓰려는 데에는...,문화,그레고리 헨더슨과 지적 동반 여행을 떠난 까닭 이현휘 한국교원대 인문사회과학교육연...,5.0
8890,8891,전문 대통령 상하원 합동회의 연설문,윤석열 대통령은 일현지시각 오전 미 상하원 합동회의에서 연설을 했다 한국 대통령의 ...,정치,전문 대통령 상하원 합동회의 연설문 윤석열 대통령은 일현지시각 오전 미 상하원 합...,2.0
